<a href="https://colab.research.google.com/github/swapnalisingh13/LLM_MiniProj/blob/main/Extract_LLM_565.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google-generativeai

In [ ]:
import google.generativeai as genai

## SET API KEY

In [ ]:
# Used to securely store your API key
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

## LIST OF MODELS

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)


models/gemini-1.0-pro-latest
models/gemini-1.0-pro
models/gemini-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro
models/gemini-1.5-pro-exp-0801
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash
models/gemini-1.5-flash-001-tuning


In [ ]:
# Model Configuration
MODEL_CONFIG = {
  "temperature": 0.2,
  "top_p": 1,
  "top_k": 32,
  "max_output_tokens": 4096,
}

## Safety Settings of Model
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  }
]

## LOAD GEMINI MODEL WITH MODEL CONFIGURATIONS

In [ ]:
model = genai.GenerativeModel(model_name = "gemini-1.5-flash",
                              generation_config = MODEL_CONFIG,
                              safety_settings = safety_settings)

## DEFINE IMAGE FORMAT TO INPUT IN GEMINI

In [ ]:
from pathlib import Path

def image_format(image_path):
    img = Path(image_path)

    if not img.exists():
        raise FileNotFoundError(f"Could not find image: {img}")

    image_parts = [
        {
            "mime_type": "image/png", ## Mime type are PNG - image/png. JPEG - image/jpeg. WEBP - image/webp
            "data": img.read_bytes()
        }
    ]
    return image_parts


## GEMINI MODEL OUTPUT

In [ ]:
def gemini_output(image_path, system_prompt, user_prompt):

    image_info = image_format(image_path)
    input_prompt= [system_prompt, image_info[0], user_prompt]
    response = model.generate_content(input_prompt)
    return response.text

### EXTRACTING PART OF THE INFORMATION FROM INVOICE

In [ ]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """

image_path = "/content/invoice.png"

user_prompt = "What is the balance amount in the image?"

gemini_output(image_path, system_prompt, user_prompt)

'The balance amount is $154.06.'

## EXTRACTING WHOLE DATA IN JSON FROM INVOICE

In [ ]:
system_prompt = """
               You are a specialist in comprehending receipts.
               Input images in the form of receipts will be provided to you,
               and your task is to respond to questions based on the content of the input image.
               """
#system_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "
image_path = "/content/invoice.png"
user_prompt = "Convert Invoice data into json format with appropriate json tags as required for the data in image "

In [ ]:
output = gemini_output(image_path, system_prompt, user_prompt)

In [ ]:
from IPython.display import Markdown
Markdown(output)

```json
{"BILL TO": "John Smith\n2 Court Square\nNew York, NY 12210", "SHIP TO": "John Smith\n3787 Pineview Drive\nCambridge, MA 12210", "INVOICE #": "US-001", "INVOICE DATE": "11/02/2019", "P.O.#": "2312/2019", "DUE DATE": "26/02/2019", "QTY": "1", "DESCRIPTION": "Front and rear brake cables", "UNIT PRICE": "100.00", "AMOUNT": "100.00", "QTY": "2", "DESCRIPTION": "New set of pedal arms", "UNIT PRICE": "15.00", "AMOUNT": "30.00", "QTY": "3", "DESCRIPTION": "Labor 3hrs", "UNIT PRICE": "5.00", "AMOUNT": "15.00", "Subtotal": "145.00", "Sales Tax 6.25%": "9.06", "TOTAL": "154.06"}
```